In [1]:
import os
import cv2
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# 이미지와 레이블 데이터를 저장할 리스트
images = []
labels = []

# 데이터 디렉토리 설정
cropped_image_dir = r"D:\data\korean 01 data"
json_dir = r"D:\data\korean skin data\open data\data\Training\Label data\digiter cam"

# 데이터 로드
for id_folder in os.listdir(cropped_image_dir):
    image_folder_path = os.path.join(cropped_image_dir, id_folder)
    json_file_path = os.path.join(json_dir, id_folder, f"{id_folder}_01_F_01.json")

    if not os.path.exists(json_file_path):
        continue

    with open(json_file_path, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    for image_name in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (128, 128))  # 예시로 128x128로 크기 조정
        images.append(image)

        # 레이블 추출 (예: 이마의 색소침착)
        label = json_data['annotations']['forehead_pigmentation']
        labels.append(label)

# 이미지와 레이블을 NumPy 배열로 변환
images = np.array(images, dtype='float32') / 255.0  # 정규화
labels = np.array(labels)

# 레이블 원-핫 인코딩
labels = to_categorical(labels, num_classes=6)  

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 모델 설계
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(6, activation='softmax')
])

# 모델 컴파일

model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"테스트 정확도: {test_acc}")






Epoch 1/10


22/22 [==============================] - 7s 277ms/step - loss: 1.3525 - accuracy: 0.4854 - val_loss: 1.2416 - val_accuracy: 0.5174
Epoch 2/10
22/22 [==============================] - 6s 259ms/step - loss: 1.2186 - accuracy: 0.5160 - val_loss: 1.1789 - val_accuracy: 0.5174
Epoch 3/10
22/22 [==============================] - 6s 268ms/step - loss: 1.2232 - accuracy: 0.4942 - val_loss: 1.4304 - val_accuracy: 0.3081
Epoch 4/10
22/22 [==============================] - 6s 255ms/step - loss: 1.2885 - accuracy: 0.4883 - val_loss: 1.1764 - val_accuracy: 0.5174
Epoch 5/10
22/22 [==============================] - 6s 254ms/step - loss: 1.1036 - accuracy: 0.5204 - val_loss: 1.2133 - val_accuracy: 0.4012
Epoch 6/10
22/22 [==============================] - 5s 244ms/step - loss: 1.0984 - accuracy: 0.5233 - val_loss: 1.2706 - val_accuracy: 0.5407
Epoch 7/10
22/22 [==============================] - 5s 234ms/step - loss: 1.1371 - accuracy: 0.5496 - val_loss: 1.1151 - val_accuracy: 0.5407


In [3]:
# 레이블 값 확인
unique_labels = np.unique(labels)
print(f"레이블 값들: {unique_labels}")

# 최대 레이블 값에 따라 num_classes 설정
num_classes = len(unique_labels)  # 레이블 값 개수를 num_classes로 설정

# 레이블 원-핫 인코딩
labels = to_categorical(labels, num_classes=num_classes)


레이블 값들: [0. 1.]


In [23]:
import os
import cv2
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [5]:
cropped_image_dir = r"D:\data\korean 01 data"
json_dir = r"D:\data\korean skin data\open data\data\Training\Label data\digiter cam"

In [18]:
# 이미지 로드
def load_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.resize(img , (224,224))
    img = img / 255.0
    return img

In [33]:
# json 파일 로드
def load_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    age = data['info']['age']
    gender = 1 if data['info']['gender'] =='F' else 0 
    skin_type = data['info']['skin_type']
    sensitive = data['info']['sensitive']

    pigmentation = data['annotations']['forehead_pigmentation']
    
    return age, gender, skin_type, sensitive, pigmentation

In [49]:
from sklearn.model_selection import train_test_split

image_dir = r"D:\data\korean 01 data"
json_dir = r"D:\data\korean skin data\open data\data\Training\Label data\digiter cam"

images = []
metadata = []

for folder_name in os.listdir(image_dir):
    folder_path = os.path.join(image_dir, folder_name)
    if os.path.isdir(folder_path):  # 폴더인지 확인
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):  # .jpg 파일만 처리
                image_path = os.path.join(folder_path, filename)
                
                # JSON 파일명 생성: 'cropped_' 제거하고 '.jpg' -> '.json'
                json_filename = filename.replace('cropped_', '').replace('.jpg', '')
                json_path = os.path.join(json_dir, folder_name, json_filename)  # JSON 파일 경로
                
                # 이미지와 JSON 로드
                image = load_image(image_path)
                if image is None:
                    continue  # 이미지 로드 실패시 건너뛰기
                age, gender, skin_type, sensitive, pigmentation = load_json(json_path)
                
                images.append(image)  # images 리스트에 추가
                metadata.append([age, gender, skin_type, sensitive, pigmentation])  # metadata 리스트에 추가

# 리스트를 numpy 배열로 변환
images = np.array(images)
metadata = np.array(metadata)

X = images
y = metadata[:,-1] # target

X_metadata = metadata[:, :-1] # pigment 제외한 나머지 데이터


X_train,X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state = 42)

X_train_images = X_train
X_val_images = X_val

X_train_metadata = X_metadata[:len(X_train)]
X_val_metadata = X_metadata[len(X_train):]

In [50]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input


image_input = Input(shape=(224,224,3), name= 'image_input')
x = layers.Conv2D(32,(3,3), activation = 'relu')(image_input)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(64,(3,3), activation = 'relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)


metadata_input = Input(shape=(4,), name='metadata_input')
y = layers.Dense(32, activation ='relu')(metadata_input)
y = layers.Dense(16, activation='relu')(y)

combined = layers.concatenate([x,y])
output = layers.Dense(6, activation='softmax')(combined)

model = models.Model(inputs =[image_input, metadata_input], outputs=output)
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image_input (InputLayer)    [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_16 (Conv2D)          (None, 222, 222, 32)         896       ['image_input[0][0]']         
                                                                                                  
 max_pooling2d_15 (MaxPooli  (None, 111, 111, 32)         0         ['conv2d_16[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_17 (Conv2D)          (None, 109, 109, 64)         18496     ['max_pooling2d_15[0][0]

In [51]:
# 모델 학습
history = model.fit(
    {'image_input': X_train_images, 'metadata_input': X_train_metadata},  # 학습 데이터
    y_train,  # 실제 레이블 (pigmentation 값)
    epochs=20,  # 에포크 수
    validation_data=(
        {'image_input': X_val_images, 'metadata_input': X_val_metadata},  # 검증 데이터
        y_val  # 검증 데이터의 실제 레이블 (pigmentation 값)
    )
)

# 학습이 완료되면 학습 과정에서의 손실과 메트릭스를 확인할 수 있습니다.
print("Training History: ", history.history)


Epoch 1/20
22/22 [==============================] - 16s 675ms/step - loss: 3.1633 - accuracy: 0.4082 - val_loss: 1.4000 - val_accuracy: 0.3721
Epoch 2/20
22/22 [==============================] - 14s 641ms/step - loss: 1.2670 - accuracy: 0.4840 - val_loss: 1.2315 - val_accuracy: 0.5174
Epoch 3/20
22/22 [==============================] - 14s 627ms/step - loss: 1.2156 - accuracy: 0.5044 - val_loss: 1.1834 - val_accuracy: 0.5174
Epoch 4/20
22/22 [==============================] - 14s 630ms/step - loss: 1.1590 - accuracy: 0.5117 - val_loss: 1.1690 - val_accuracy: 0.5291
Epoch 5/20
22/22 [==============================] - 14s 632ms/step - loss: 1.1325 - accuracy: 0.5233 - val_loss: 1.1526 - val_accuracy: 0.5116
Epoch 6/20
22/22 [==============================] - 14s 628ms/step - loss: 1.1206 - accuracy: 0.5175 - val_loss: 1.1677 - val_accuracy: 0.4884
Epoch 7/20
22/22 [==============================] - 14s 638ms/step - loss: 1.1092 - accuracy: 0.5015 - val_loss: 1.1567 - val_accuracy: 0.5233

In [52]:
# 모델 평가
test_loss, test_accuracy = model.evaluate(
    {'image_input': X_val_images, 'metadata_input': X_val_metadata},  # 검증 데이터
    y_val  # 검증 데이터의 실제 레이블
)

print("Test Loss: ", test_loss)
print("Test Accuracy: ", test_accuracy)


6/6 [==============================] - 1s 112ms/step - loss: 1.0900 - accuracy: 0.5349
Test Loss:  1.0899670124053955
Test Accuracy:  0.5348837375640869
